In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the dataset
df = pd.read_csv("sampled_dataset.csv")

# Keep only necessary columns
df = df[['text', 'target']]

# Convert target: 0 -> 0 (negative), 4 -> 1 (positive)
df['label'] = df['target'].map({0: 0, 4: 1})

# Vectorize text using TF-IDF (limited to 1000 features)
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['text'])
y = df['label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Random Forest with fixed, optimized settings
rf = RandomForestClassifier(
    n_estimators=50,      # fewer trees for faster training
    max_depth=10,         # limit depth to control size/speed
    min_samples_split=5,  # slightly increase to reduce overfitting
    random_state=42,
    n_jobs=-1             # use all available CPU cores
)
rf.fit(X_train, y_train)

# Evaluate on test data
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

# Predict new samples
new_texts = [
    "I really enjoyed this, highly recommend!",
    "Completely useless, very disappointed."
]

new_vectors = vectorizer.transform(new_texts)
predictions = rf.predict(new_vectors)

for text, pred in zip(new_texts, predictions):
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Text: '{text}' => Sentiment: {sentiment}")


Accuracy: 0.6648333333333334

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.48      0.59      6000
           1       0.62      0.85      0.72      6000

    accuracy                           0.66     12000
   macro avg       0.69      0.66      0.65     12000
weighted avg       0.69      0.66      0.65     12000

Text: 'I really enjoyed this, highly recommend!' => Sentiment: Negative
Text: 'Completely useless, very disappointed.' => Sentiment: Positive
